In [1]:
! echo 'hello'

hello


In [2]:
! cd /Users/equaglio/Decider/decider-genetics
! rm -f poetry.lock # remove any `poetry.lock` file already present in the folder
! poetry env use /Users/equaglio/.pyenv/versions/3.13.2/bin/python3.13 # if current python version is not compatible, you can install a version 3.10+
! poetry install

Using virtualenv: /Users/equaglio/Library/Caches/pypoetry/virtualenvs/decider-genetics-QA_7G8QC-py3.13
Updating dependencies
Resolving dependencies... (3.1s)

No dependencies to install or update

Writing lock file

Installing the current project: decider-genetics (0.1.3)Installing the current project: decider-genetics (0.1.3)


In [3]:
! poetry update

Updating dependencies
Resolving dependencies... (0.8s)

No dependencies to install or update


In [4]:
! eval $(poetry env activate) # new implementation of `poetry shell`
! poetry env info


Virtualenv
Python:         3.13.2
Implementation: CPython
Path:           /Users/equaglio/Library/Caches/pypoetry/virtualenvs/decider-genetics-QA_7G8QC-py3.13
Executable:     /Users/equaglio/Library/Caches/pypoetry/virtualenvs/decider-genetics-QA_7G8QC-py3.13/bin/python
Valid:          True

Base
Platform:   darwin
OS:         posix
Python:     3.13.2
Path:       /Users/equaglio/.pyenv/versions/3.13.2
Executable: /Users/equaglio/.pyenv/versions/3.13.2/bin/python3.13


In [5]:
! pwd
! which python
! ls
! poetry check

/Users/equaglio/Decider/decider-genetics
/Users/equaglio/.pyenv/shims/python
Dockerfile                data                      old
LICENSE                   decider_genetics          poetry.lock
README.md                 docker                    pyproject.toml
biocypher-log             docker-compose-next.yml   scripts
biocypher-out             docker-compose.yml        vignette.ipynb
config                    docker-variables.env
create_knowledge_graph.py import.report
If you want to set the version dynamically via `poetry build --local-version` or you are using a plugin, which sets the version dynamically, you should define the version in [tool.poetry] and add 'version' to [project.dynamic].
If you want to define multiple readmes, you should define them in [tool.poetry] and add 'readme' to [project.dynamic].


In [6]:
! poetry show

alabaster                     1.0.0       A light, configurable Sphinx theme
annotated-types               0.7.0       Reusable constraint types to use ...
appdirs                       1.4.4       A small Python module for determi...
appnope                       0.1.4       Disable App Nap on macOS >= 10.9
asttokens                     3.0.0       Annotate AST trees with source co...
attrs                         25.3.0      Classes Without Boilerplate
babel                         2.17.0      Internationalization utilities
biocypher                     0.9.5       A unifying framework for biomedic...
black                         25.1.0      The uncompromising code formatter.
certifi                       2025.4.26   Python package for providing Mozi...
chardet                       5.2.0       Universal encoding detector for P...
charset-normalizer            3.4.2       The Real First Universal Charset ...
click                         8.2.0       Composable command line interface

In [7]:
# mapping files
! ls -R decider_genetics/adapters/

my_mapping_1_variants.yaml
my_mapping_1_variants_solve_inverse_property_patients.yaml
my_mapping_1_variants_solve_inverse_property_variants.yaml
my_mapping_2_clinical.yaml
my_mapping_2_variants.yaml
my_mapping_3_clinical.yaml
my_mapping_3_cna.yaml
my_mapping_3_variants.yaml


In [8]:
import sys
print(sys.executable)
! which python
#sys.path.append("/Users/equaglio/Decider/Ontoweaver/src")

/Users/equaglio/Library/Caches/pypoetry/virtualenvs/decider-genetics-QA_7G8QC-py3.13/bin/python
/Users/equaglio/.pyenv/shims/python


In [9]:
! poetry add ipykernel

The following packages are already present in the pyproject.toml and will be skipped:

  - ipykernel

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [10]:
# to point the kernel of the notebook to the poetry environment activated. 
! poetry run python -m ipykernel install --user --name=my-poetry-env --display-name "decider genetics poetry env"

Installed kernelspec my-poetry-env in /Users/equaglio/Library/Jupyter/kernels/my-poetry-env


### /<u>!</u>\  Now change kernel and make sure to use "decider genetics poetry env" before running next cell

In [3]:
import argparse
import logging
import pandas as pd
import yaml
from biocypher import BioCypher
import ontoweaver

In [4]:
# tables in data/
! ls -R data

oncodash files                         synthetic_cna.csv
oncokb_biomarker_drug_associations.tsv synthetic_variants.csv
synthetic_clinical.csv

data/oncodash files:
BiologicalProcess-part000.csv       GeneToBiologicalProcess-part000.csv


In [5]:
variants_df=pd.read_table('data/synthetic_variants.csv')
variants_df.head(n=5)

,patient,CHROM,POS,REF,ALT,ID,FILTER,cytoBand,Func.MANE,Gene.MANE,...,gnomAD_genome_FIN,gnomAD_genome_max,gnomAD_exome_nc_ALL,gnomAD_exome_nc_NFE,gnomAD_exome_nc_NFE_SWE,gnomAD_exome_nc_FIN,gnomAD_exome_nc_max,Truncal,readCounts,samples
0,patient1,chr8,32660776,T,A,.,PASS,8p12,intronic,NRG1,...,.,.,.,.,.,.,.,heterogeneous,"38,0;21,0;26,13;37,1",patient1_Per
1,patient1,chr20,40688333,G,C,.,PASS,20q12,exonic,MAFB,...,.,.,.,.,.,.,.,homogeneous,"65,0;70,17;30,5;62,9;58,7",patient1_Lum;patient1_Per
2,patient1,chr2,197391056,CAGTAAA,C,.,PASS,2q33.1,UTR3,SF3B1,...,.,.,.,.,.,.,.,heterogeneous,"30,0;29,17;77,0;60,0;44,0",patient1_Per;patient1_Ome
3,patient1,chrX,124089869,G,T,.,PASS,Xq25,intronic,STAG2,...,.,.,.,.,.,.,.,heterogeneous,"34,0;33,0;45,0;22,3;28,7",patient1_Asc;patient1_Lum
4,patient1,chr6,151916206,G,A,rs977154062,PASS,6q25.1,intronic,ESR1,...,0,9.52064e-05,.,.,.,.,.,homogeneous,"58,0;21,6",patient1_Lum;patient1_Asc;patient1_Per;patient...


In [6]:
variants_df.columns

Index(['patient', 'CHROM', 'POS', 'REF', 'ALT', 'ID', 'FILTER', 'cytoBand',
       'Func.MANE', 'Gene.MANE', 'GeneDetail.MANE', 'ExonicFunc.MANE',
       'AAChange.MANE', 'Func.refGene', 'Gene.refGene', 'GeneDetail.refGene',
       'ExonicFunc.refGene', 'AAChange.refGene', 'genomicSuperDups',
       'dbscSNV_ADA_SCORE', 'dbscSNV_RF_SCORE', 'COSMIC_ID',
       'COSMIC_OCCURRENCE', 'COSMIC_TOTAL_OCC', 'COSMIC_CONF_SOMA', 'CLNSIG',
       'CLNSIGCONF', 'CLNDN', 'CLNREVSTAT', 'CLNALLELEID', 'CLNDISDB',
       'Interpro_domain', 'regulomeDB', 'CADD_raw', 'CADD_phred', '1000G_ALL',
       '1000G_EUR', 'gnomAD_genome_ALL', 'gnomAD_genome_NFE',
       'gnomAD_genome_FIN', 'gnomAD_genome_max', 'gnomAD_exome_nc_ALL',
       'gnomAD_exome_nc_NFE', 'gnomAD_exome_nc_NFE_SWE', 'gnomAD_exome_nc_FIN',
       'gnomAD_exome_nc_max', 'Truncal', 'readCounts', 'samples'],
      dtype='object')

### Here we show how to write a proper schema config

### Here we show how to write a proper mapping
TODO: highlight and show problem of consistency of direction between predicate as defined in the ontology (pointed to in the biocypher_config file) and subject-object assignment in the ontoweaver mapping:
    
    Issues:
* apparently biocypher has no way to declare inverse property
* it's not clear to me if and why (conceptually) we shouldn't be able to treat rowIndex as object and another column as subject (other than forcing the user to always have a subject when they define the mapping).
* we could add an inverse property on our ontology for example with protege as we create subclasses through biocypher, but personalizing the ontology through 2 different tools is not as convenient/easy.

In [7]:
# Edit the biocypher config to point to the new schema_config file

with open('config/biocypher_config.yaml', 'r') as f:
    bc = yaml.safe_load(f)

bc['biocypher']['schema_config_path'] = 'schema_config_1.yaml'

with open('config/biocypher_config.yaml', 'w') as f:
    yaml.dump(bc, f, default_flow_style=False)

In [8]:
#    #import decider_genetics.adapters.preprocessing_dataframes as preprocess_df
#    #import decider_genetics.adapters.preprocessing_variants as preprocess_snv
#    
#    
#    bc = BioCypher(
#        biocypher_config_path="config/biocypher_config.yaml",
#        schema_config_path="config/schema_config_1.yaml"
#    )
#    
#    # Actually extract data.
#    nodes = []
#    edges = []
#    
#    data_mappings = {}
#    
#    logging.info(f"Weave synthetic variants data")
#    
#    #variants_df = pd.read_csv('data/', sep='\t')
#    
#    mapping_file = "./decider_genetics/adapters/my_mapping_1_variants.yaml"
#    with open(mapping_file) as fd:
#        conf = yaml.full_load(fd)
#    
#    adapter = ontoweaver.tabular.extract_table(df=variants_df, config=conf,separator = None, affix= "none")
#    
#    nodes += adapter.nodes
#    edges += adapter.edges
#    
#    logging.info(f"Wove variants: {len(nodes)} nodes, {len(edges)} edges.")

#    # Write everything.
#    n, e = ontoweaver.extract(data_mappings)
#    nodes += n
#    edges += e

#    import_file = ontoweaver.reconciliate_write(nodes, edges, "config/biocypher_config.yaml", "config/schema_config_1.yaml", separator=", ")
#    # bc.write_schema_info(as_node=True)
# 
#    print(import_file)

## Here below, an attempt to force Ontoweaver to set rowIndex as object (but it doesn't work so new fea)

In [9]:
#    #import decider_genetics.adapters.preprocessing_dataframes as preprocess_df
#    #import decider_genetics.adapters.preprocessing_variants as preprocess_snv
#    
#    
#    bc = BioCypher(
#        biocypher_config_path="config/biocypher_config.yaml",
#        schema_config_path="config/schema_config_1.yaml"
#    )
#    
#    # Actually extract data.
#    nodes = []
#    edges = []
#    
#    data_mappings = {}
#    
#    logging.info(f"Weave synthetic variants data")
#    
#    #variants_df = pd.read_csv('data/', sep='\t')
#    
#    
#    # Patient to SequenceVariant
#    mapping_file = "./decider_genetics/adapters/my_mapping_1_variants_solve_inverse_property_patients.yaml"
#    with open(mapping_file) as fd:
#        conf = yaml.full_load(fd)
#    
#    adapter = ontoweaver.tabular.extract_table(df=variants_df, config=conf,separator = None, affix= "none")
#    
#    nodes += adapter.nodes
#    edges += adapter.edges
#    
#    
#    # SequenceVariant to Gene
#    mapping_file = "./decider_genetics/adapters/my_mapping_1_variants_solve_inverse_property_variants.yaml"
#    with open(mapping_file) as fd:
#        conf = yaml.full_load(fd)
#    
#    adapter = ontoweaver.tabular.extract_table(df=variants_df, config=conf,separator = None, affix= "none")
#    
#    nodes += adapter.nodes
#    edges += adapter.edges
#    
#    logging.info(f"Wove variants: {len(nodes)} nodes, {len(edges)} edges.")
#    
#    # # Write everything.
#    # n, e = ontoweaver.extract(data_mappings)
#    # nodes += n
#    # edges += e

#    import_file = ontoweaver.reconciliate_write(nodes, edges, "config/biocypher_config.yaml", "config/schema_config_1.yaml", separator=", ")
#    # bc.write_schema_info(as_node=True)
#    
#    print(import_file)

## Here, we momentarily neglect the relationship direction issue, and run ontoweaver on the first dataset with no properties.

In [10]:
#import decider_genetics.adapters.preprocessing_dataframes as preprocess_df
#import decider_genetics.adapters.preprocessing_variants as preprocess_snv


bc = BioCypher(
    biocypher_config_path="config/biocypher_config.yaml",
    schema_config_path="config/schema_config_1.yaml"
)

# Actually extract data.
nodes = []
edges = []

data_mappings = {}

logging.info(f"Weave synthetic variants data")

#variants_df = pd.read_csv('data/', sep='\t')


# SequenceVariant to Gene
mapping_file = "./decider_genetics/adapters/my_mapping_1_variants.yaml"
with open(mapping_file) as fd:
    conf = yaml.full_load(fd)

adapter = ontoweaver.tabular.extract_table(df=variants_df, config=conf,separator = None, affix= "none")

nodes += adapter.nodes
edges += adapter.edges

logging.info(f"Wove variants: {len(nodes)} nodes, {len(edges)} edges.")

# # Write everything.
# n, e = ontoweaver.extract(data_mappings)
# nodes += n
# edges += e

import_file = ontoweaver.reconciliate_write(nodes, edges, "config/biocypher_config.yaml", "config/schema_config_1.yaml", separator=", ")
# bc.write_schema_info(as_node=True)

print(import_file)

INFO -- Running BioCypher with schema configuration from config/schema_config_1.yaml.
INFO -- Running BioCypher with schema configuration from config/schema_config_1.yaml.
INFO:biocypher:Running BioCypher with schema configuration from config/schema_config_1.yaml.
INFO -- Loading ontologies...
INFO:biocypher:Loading ontologies...
INFO -- Instantiating OntologyAdapter class for https://github.com/biolink/biolink-model/raw/v3.2.1/biolink-model.owl.ttl.
INFO:biocypher:Instantiating OntologyAdapter class for https://github.com/biolink/biolink-model/raw/v3.2.1/biolink-model.owl.ttl.
INFO -- Creating output directory `/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533`.
INFO:biocypher:Creating output directory `/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533`.
DEBUG:biocypher:Writing node CSV from generator.
DEBUG:biocypher:Translating nodes to BioCypher
DEBUG:biocypher:Finished translating nodes to BioCypher.
INFO -- Writing 650 entries to SequenceVa

/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533/neo4j-admin-import-call.sh


In [11]:
! pwd

/Users/equaglio/Decider/decider-genetics


In [12]:
! ls -R biocypher-out

20250426162526 20250429174419 20250509162452 20250512142218 20250512175232
20250426173219 20250429183224 20250509175856 20250512144931 20250512175530
20250429165625 20250429183405 20250509180051 20250512150449 20250512182645
20250429170857 20250429183537 20250511124514 20250512152325 20250512182812
20250429172231 20250429195129 20250512140048 20250512155830 20250512183622
20250429174307 20250429200648 20250512140248 20250512174434 20250512184533
20250429174328 20250429200832 20250512140830 20250512175038
20250429174351 20250509152553 20250512141620 20250512175043

biocypher-out/20250426162526:
Patient-header.csv                   SequenceVariant-part000.csv
Patient-part000.csv                  VariantToGeneAssociation-header.csv
PatientHasVariant-header.csv         VariantToGeneAssociation-part000.csv
PatientHasVariant-part000.csv        neo4j-admin-import-call.sh
SequenceVariant-header.csv           neo4j-admin-import-call.sh-e

biocypher-out/20250426173219:
Patient-header.csv        

In [18]:
! neo4j-admin server stop

Stopping Neo4j.... stopped.


In [19]:
! neo4j stop

Neo4j is not running.


In [15]:
# DO: substitute timestamp folder with the (last) one in `biocypher-out`
! sed -i -e 's,bin/neo4j-admin,neo4j-admin,g' biocypher-out/20250512184533/neo4j-admin-import-call.sh  

In [16]:
! sh biocypher-out/20250512184533/neo4j-admin-import-call.sh

Starting to import, output will be saved to: /Users/equaglio/neo4j-community-2025.03.0/logs/neo4j-admin-import-2025-05-12.18.45.57.log
Neo4j version: 2025.03.0
Importing the contents of these files into /Users/equaglio/neo4j-community-2025.03.0/data/databases/test:
Nodes:
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533/SequenceVariant-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533/SequenceVariant-part000.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533/Gene-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533/Gene-part000.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533/Patient-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533/Patient-part000.csv

Relationships:
  null:
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512184533/PatientHasVariant-header.csv
  /Users/equaglio/Decider/decider-genetics/

In [17]:
#! neo4j-admin server start # this for some reason does not work
! neo4j console # this one does

Directories in use:
home:         /Users/equaglio/neo4j-community-2025.03.0
config:       /Users/equaglio/neo4j-community-2025.03.0/conf
logs:         /Users/equaglio/neo4j-community-2025.03.0/logs
plugins:      /Users/equaglio/neo4j-community-2025.03.0/plugins
import:       /Users/equaglio/neo4j-community-2025.03.0/import
data:         /Users/equaglio/neo4j-community-2025.03.0/data
certificates: /Users/equaglio/neo4j-community-2025.03.0/certificates
licenses:     /Users/equaglio/neo4j-community-2025.03.0/licenses
run:          /Users/equaglio/neo4j-community-2025.03.0/run
Starting Neo4j.
2025-05-12 16:46:00.339+0000 INFO  Logging config in use: File '/Users/equaglio/neo4j-community-2025.03.0/conf/user-logs.xml'
2025-05-12 16:46:00.345+0000 INFO  Starting...
2025-05-12 16:46:00.901+0000 INFO  This instance is ServerId{865d2652} (865d2652-49d9-4958-b8e4-33516e9c653f)
2025-05-12 16:46:01.482+0000 INFO  ======== Neo4j 2025.03.0 ========
2025-05-12 16:46:02.398+0000 INFO  Anonymous Usage D

### In neo4j, use `MATCH (p)<-[r]-(v)-[l]->(g) RETURN p,r,v,l,g;` to visualize (part of) the SKG highlighting patient-variant-gene connection.

## /<u>!</u>\ Interrupt execution of previous block to gently shutdown neo4j server

In [20]:
! neo4j status # to check, after some seconds, the effect of shutdown

Neo4j is not running.
Run with '--verbose' for a more detailed error message.


In [ ]:
# in case shut down didnt work, run 
! ps aux | grep neo4j
# and then 
# kill <PID>
# where <PID> is the process identifier of the neo4j session

## Now let's add properties to those nodes

Patients properties are stored in `data/synthetic_clinical.csv`

In [21]:
clinical_df = pd.read_table('data/synthetic_clinical.csv', sep=';')
clinical_df.head(n=5)

,Patient,Age,BMI,Stage,Treatment,Chemotherapy cycles,Maintenance,PARPi,Primary outcome,BRCA mutation,HR deficient
0,patient1,72,21.484375,IVB,PDS,6.0,No,No,Partial Response,NaN,HRD negative
1,patient2,64,28.628792,IIB,PDS,6.0,NaN,No,Partial Response,Yes,NaN
2,patient3,70,26.838648,IVA,NACT,6.0,No,Yes,Partial Response,NaN,NaN
3,patient4,78,23.054562,IVA,NACT,6.0,Letrozole,No,Partial Response,NaN,HRD positive
4,patient5,75,20.703125,IVB,PDS,6.0,No,No,Progressive Disease,NaN,HRD negative


### We see there was inconsistence of decimal separator between BMI (comma) and Chemotherapy cycles (dot). This originates an error later on in neo4j so i set comma everywhere. Should be automatized since we are already specifying type of `BMI` in `schema_config.yaml` as `float`  

In [24]:
# Edit the biocypher config to point to the new schema_config file

with open('config/biocypher_config.yaml', 'r') as f:
    bc = yaml.safe_load(f)

bc['biocypher']['schema_config_path'] = 'schema_config_2.yaml'

with open('config/biocypher_config.yaml', 'w') as f:
    yaml.dump(bc, f, default_flow_style=False)

#! cat config/biocypher_config.yaml

In [25]:
#import decider_genetics.adapters.preprocessing_dataframes as preprocess_df
#import decider_genetics.adapters.preprocessing_variants as preprocess_snv


bc = BioCypher(
    biocypher_config_path="config/biocypher_config.yaml",
    schema_config_path="config/schema_config_2.yaml"
)

# Actually extract data.
nodes = []
edges = []

data_mappings = {}

logging.info(f"Weave synthetic variants data")

#variants_df = pd.read_csv('data/', sep='\t')

mapping_file = "./decider_genetics/adapters/my_mapping_2_variants.yaml"
with open(mapping_file) as fd:
    conf = yaml.full_load(fd)

adapter = ontoweaver.tabular.extract_table(df=variants_df, config=conf,separator = "\t", affix= "none")

nodes += adapter.nodes
edges += adapter.edges

logging.info(f"Wove variants: {len(nodes)} nodes, {len(edges)} edges.")



logging.info(f"Weave synthetic clinical data...")

mapping_file = "./decider_genetics/adapters/my_mapping_2_clinical.yaml"
with open(mapping_file) as fd:
    conf = yaml.full_load(fd)

adapter = ontoweaver.tabular.extract_table(df=clinical_df, config=conf,separator = "\t", affix= "none")

nodes += adapter.nodes
edges += adapter.edges

logging.info(f"Wove Clinical: {len(nodes)} nodes, {len(edges)} edges.")


# # Write everything.
# n, e = ontoweaver.extract(data_mappings)
# nodes += n
# edges += e

import_file = ontoweaver.reconciliate_write(nodes, edges, "config/biocypher_config.yaml", "config/schema_config_2.yaml", separator=", ")
# bc.write_schema_info(as_node=True)

print(import_file)


INFO -- Running BioCypher with schema configuration from config/schema_config_2.yaml.
INFO:biocypher:Running BioCypher with schema configuration from config/schema_config_2.yaml.
INFO -- Running BioCypher with schema configuration from config/schema_config_2.yaml.
INFO:biocypher:Running BioCypher with schema configuration from config/schema_config_2.yaml.
INFO -- Loading ontologies...
INFO:biocypher:Loading ontologies...
INFO -- Instantiating OntologyAdapter class for https://github.com/biolink/biolink-model/raw/v3.2.1/biolink-model.owl.ttl.
INFO:biocypher:Instantiating OntologyAdapter class for https://github.com/biolink/biolink-model/raw/v3.2.1/biolink-model.owl.ttl.
INFO -- Creating output directory `/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059`.
INFO:biocypher:Creating output directory `/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059`.
DEBUG:biocypher:Writing node CSV from generator.
DEBUG:biocypher:Translating nodes to BioCypher
DEBUG

/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059/neo4j-admin-import-call.sh


TO FIX!!! The previous attempt gives error, either with or without properties (tried cos there was apparently a bug in property parsing but later was fixed and i pulled onto my local ontoweaver repo so it shoud not be the problem...)

Same error also in the third schema version. 

In [26]:
! ls -R biocypher-out

20250426162526 20250429174419 20250509162452 20250512142218 20250512175232
20250426173219 20250429183224 20250509175856 20250512144931 20250512175530
20250429165625 20250429183405 20250509180051 20250512150449 20250512182645
20250429170857 20250429183537 20250511124514 20250512152325 20250512182812
20250429172231 20250429195129 20250512140048 20250512155830 20250512183622
20250429174307 20250429200648 20250512140248 20250512174434 20250512184533
20250429174328 20250429200832 20250512140830 20250512175038 20250512184954
20250429174351 20250509152553 20250512141620 20250512175043 20250512185059

biocypher-out/20250426162526:
Patient-header.csv                   SequenceVariant-part000.csv
Patient-part000.csv                  VariantToGeneAssociation-header.csv
PatientHasVariant-header.csv         VariantToGeneAssociation-part000.csv
PatientHasVariant-part000.csv        neo4j-admin-import-call.sh
SequenceVariant-header.csv           neo4j-admin-import-call.sh-e

biocypher-out/202504261732

In [27]:
# DO: substitute timestamp folder with the (last) one in `biocypher-out`
! sed -i -e 's,bin/neo4j-admin,neo4j-admin,g' biocypher-out/20250512185059/neo4j-admin-import-call.sh

In [28]:
! sh biocypher-out/20250512185059/neo4j-admin-import-call.sh --verbose

Starting to import, output will be saved to: /Users/equaglio/neo4j-community-2025.03.0/logs/neo4j-admin-import-2025-05-12.18.51.24.log
Neo4j version: 2025.03.0
Importing the contents of these files into /Users/equaglio/neo4j-community-2025.03.0/data/databases/test:
Nodes:
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059/Gene-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059/Gene-part000.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059/SequenceVariant-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059/SequenceVariant-part000.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059/Patient-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059/Patient-part000.csv

Relationships:
  null:
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512185059/PatientHasVariant-header.csv
  /Users/equaglio/Decider/decider-genetics/

In [29]:
#! neo4j-admin server start # this for some reason does not work
! neo4j console # this one does

Directories in use:
home:         /Users/equaglio/neo4j-community-2025.03.0
config:       /Users/equaglio/neo4j-community-2025.03.0/conf
logs:         /Users/equaglio/neo4j-community-2025.03.0/logs
plugins:      /Users/equaglio/neo4j-community-2025.03.0/plugins
import:       /Users/equaglio/neo4j-community-2025.03.0/import
data:         /Users/equaglio/neo4j-community-2025.03.0/data
certificates: /Users/equaglio/neo4j-community-2025.03.0/certificates
licenses:     /Users/equaglio/neo4j-community-2025.03.0/licenses
run:          /Users/equaglio/neo4j-community-2025.03.0/run
Starting Neo4j.
2025-05-12 16:51:28.001+0000 INFO  Logging config in use: File '/Users/equaglio/neo4j-community-2025.03.0/conf/user-logs.xml'
2025-05-12 16:51:28.007+0000 INFO  Starting...
2025-05-12 16:51:28.536+0000 INFO  This instance is ServerId{865d2652} (865d2652-49d9-4958-b8e4-33516e9c653f)
2025-05-12 16:51:29.081+0000 INFO  ======== Neo4j 2025.03.0 ========
2025-05-12 16:51:29.975+0000 INFO  Anonymous Usage D

### In neo4j, use `MATCH (p)<-[r]-(v)-[l]->(g) RETURN p,r,v,l,g;` to visualize (part of) the SKG highlighting patient-variant-gene connection. Click on nodes to explore properties.

## And finally let's add a CopyNumberAlteration node connecting Patient and Gene as well.
Those alterations are store in Patients `data/synthetic_cns.csv`. Such tables store also information about the chromosome of location of genes present in the `synthetic_variants.csv` table, so we add it as gene property.

In [30]:
cna_df = pd.read_table('data/synthetic_cna.csv', sep='\t')
cna_df.head(n=5)

,ID,Gene,chr,start,end,strand,band,type,sample,nProbesCr,nProbesAf,logR,baf,nAraw,nBraw,nMajor,nMinor,purifiedLogR,purifiedBaf,purifiedLoh,CNstatus,LOHstatus,minPurifiedLogR,maxPurifiedLogR,breaksInGene
0,ENSG00000168685,IL7R,chr5,35852695,35879603,1,p13.2,protein_coding,patient1,7595,5440,0.9068,0.3309,9.5999,4.1648,10.0,4.0,1.1350,0.3026,0.3949,AMP,HET,1.1350,1.1350,0
1,ENSG00000119772,DNMT3A,chr2,25227855,25342590,-1,p23.3,protein_coding,patient1,80852,45350,0.0811,0.4819,2.7730,2.0427,3.0,2.0,0.3761,0.4242,0.1517,Normal,HET,0.3761,0.3761,0
2,ENSG00000068078,FGFR3,chr4,1793293,1808872,1,p16.3,protein_coding,patient1,11371,8948,0.6256,0.4086,3.2568,2.1358,3.0,2.0,0.7372,0.3961,0.2079,Normal,HET,0.7372,0.7372,0
3,ENSG00000185920,PTCH1,chr9,95442980,95517057,-1,q22.32,protein_coding,patient1,31710,11538,-0.1153,0.4152,3.1927,2.2095,3.0,2.0,-0.1234,0.4090,0.1820,Normal,HET,-0.1234,-0.1234,0
4,ENSG00000106462,EZH2,chr7,148807383,148884321,-1,q36.1,protein_coding,patient1,7948,5736,0.3978,0.4968,2.5976,2.5558,3.0,3.0,0.5297,0.4959,0.0081,Normal,HET,0.5297,0.5297,0


In this case we could use the ID as identifier, but not knowing precicely what it represents, I used the most conservative approach of using the `rowIndex` as `ID`. This raises the problem of potential collision with the `ID`s of `sequence variant` nodes, that were inserted using `rowIndex` as `ID`. That's why I now define, in the `my_mapping_3_variants.yaml` the ID of the variants as a custom concatenation of thr 4 properies `CHROM`-`POS`-`REF`-`ALT`, deemed necessary and sufficient, in the present context, to univocally define each variant. (It's an example...)

In [ ]:
# Edit the biocypher config to point to the new schema_config file

with open('config/biocypher_config.yaml', 'r') as f:
    bc = yaml.safe_load(f)

bc['biocypher']['schema_config_path'] = 'schema_config_3.yaml'

with open('config/biocypher_config.yaml', 'w') as f:
    yaml.dump(bc, f, default_flow_style=False)

#! cat config/biocypher_config.yaml

In [41]:
#import decider_genetics.adapters.preprocessing_dataframes as preprocess_df
#import decider_genetics.adapters.preprocessing_variants as preprocess_snv


bc = BioCypher(
    biocypher_config_path="config/biocypher_config.yaml",
    schema_config_path="config/schema_config_3.yaml"
)

# Actually extract data.
nodes = []
edges = []

data_mappings = {}

logging.info(f"Weave synthetic variants data")

#variants_df = pd.read_csv('data/', sep='\t')

mapping_file = "./decider_genetics/adapters/my_mapping_3_variants.yaml"
with open(mapping_file) as fd:
    conf = yaml.full_load(fd)

adapter = ontoweaver.tabular.extract_table(df=variants_df, config=conf,separator = None, affix= "none")

nodes += adapter.nodes
edges += adapter.edges

logging.info(f"Wove variants: {len(nodes)} nodes, {len(edges)} edges.")



logging.info(f"Weave synthetic clinical data...")

mapping_file = "./decider_genetics/adapters/my_mapping_3_clinical.yaml"
with open(mapping_file) as fd:
    conf = yaml.full_load(fd)

adapter = ontoweaver.tabular.extract_table(df=clinical_df, config=conf,separator = ";", affix= "none")

nodes += adapter.nodes
edges += adapter.edges

logging.info(f"Wove Clinical: {len(nodes)} nodes, {len(edges)} edges.")



logging.info(f"Weave synthetic copy number alteration data...")

mapping_file = "./decider_genetics/adapters/my_mapping_3_cna.yaml"
with open(mapping_file) as fd:
    conf = yaml.full_load(fd)

adapter = ontoweaver.tabular.extract_table(df=cna_df, config=conf,separator = None, affix= "none")

nodes += adapter.nodes
edges += adapter.edges

logging.info(f"Wove CNAs: {len(nodes)} nodes, {len(edges)} edges.")

# # Write everything.
# n, e = ontoweaver.extract(data_mappings)
# nodes += n
# edges += e

import_file = ontoweaver.reconciliate_write(nodes, edges, "config/biocypher_config.yaml", "config/schema_config_3.yaml", separator=", ")
# bc.write_schema_info(as_node=True) 

print(import_file)


INFO -- Running BioCypher with schema configuration from config/schema_config_3.yaml.
INFO:biocypher:Running BioCypher with schema configuration from config/schema_config_3.yaml.
INFO -- Running BioCypher with schema configuration from config/schema_config_3.yaml.
INFO:biocypher:Running BioCypher with schema configuration from config/schema_config_3.yaml.
INFO -- Loading ontologies...
INFO:biocypher:Loading ontologies...
INFO -- Instantiating OntologyAdapter class for https://github.com/biolink/biolink-model/raw/v3.2.1/biolink-model.owl.ttl.
INFO:biocypher:Instantiating OntologyAdapter class for https://github.com/biolink/biolink-model/raw/v3.2.1/biolink-model.owl.ttl.
INFO -- Creating output directory `/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340`.
INFO:biocypher:Creating output directory `/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340`.
DEBUG:biocypher:Writing node CSV from generator.
DEBUG:biocypher:Translating nodes to BioCypher
DEBUG

/Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340/neo4j-admin-import-call.sh


In [42]:
! ls -R biocypher-out

20250426162526 20250429183224 20250509180051 20250512152325 20250512183622
20250426173219 20250429183405 20250511124514 20250512155830 20250512184533
20250429165625 20250429183537 20250512140048 20250512174434 20250512184954
20250429170857 20250429195129 20250512140248 20250512175038 20250512185059
20250429172231 20250429200648 20250512140830 20250512175043 20250512200322
20250429174307 20250429200832 20250512141620 20250512175232 20250512200340
20250429174328 20250509152553 20250512142218 20250512175530
20250429174351 20250509162452 20250512144931 20250512182645
20250429174419 20250509175856 20250512150449 20250512182812

biocypher-out/20250426162526:
Patient-header.csv                   SequenceVariant-part000.csv
Patient-part000.csv                  VariantToGeneAssociation-header.csv
PatientHasVariant-header.csv         VariantToGeneAssociation-part000.csv
PatientHasVariant-part000.csv        neo4j-admin-import-call.sh
SequenceVariant-header.csv           neo4j-admin-import-call.sh

In [43]:
# DO: substitute timestamp folder with the (last) one in `biocypher-out`
! sed -i -e 's,bin/neo4j-admin,neo4j-admin,g' biocypher-out/20250512200340/neo4j-admin-import-call.sh

In [44]:
! sh biocypher-out/20250512200340/neo4j-admin-import-call.sh --verbose

Starting to import, output will be saved to: /Users/equaglio/neo4j-community-2025.03.0/logs/neo4j-admin-import-2025-05-12.20.05.32.log
Neo4j version: 2025.03.0
Importing the contents of these files into /Users/equaglio/neo4j-community-2025.03.0/data/databases/test:
Nodes:
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340/Gene-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340/Gene-part000.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340/Patient-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340/Patient-part000.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340/SequenceVariant-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340/SequenceVariant-part000.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/20250512200340/CopyNumberAlteration-header.csv
  /Users/equaglio/Decider/decider-genetics/biocypher-out/2025051

In [45]:
#! neo4j-admin server start # this for some reason does not work
! neo4j console # this one does

Directories in use:
home:         /Users/equaglio/neo4j-community-2025.03.0
config:       /Users/equaglio/neo4j-community-2025.03.0/conf
logs:         /Users/equaglio/neo4j-community-2025.03.0/logs
plugins:      /Users/equaglio/neo4j-community-2025.03.0/plugins
import:       /Users/equaglio/neo4j-community-2025.03.0/import
data:         /Users/equaglio/neo4j-community-2025.03.0/data
certificates: /Users/equaglio/neo4j-community-2025.03.0/certificates
licenses:     /Users/equaglio/neo4j-community-2025.03.0/licenses
run:          /Users/equaglio/neo4j-community-2025.03.0/run
Starting Neo4j.
2025-05-12 18:06:54.873+0000 INFO  Logging config in use: File '/Users/equaglio/neo4j-community-2025.03.0/conf/user-logs.xml'
2025-05-12 18:06:54.878+0000 INFO  Starting...
2025-05-12 18:06:55.504+0000 INFO  This instance is ServerId{865d2652} (865d2652-49d9-4958-b8e4-33516e9c653f)
2025-05-12 18:06:56.071+0000 INFO  ======== Neo4j 2025.03.0 ========
2025-05-12 18:06:56.982+0000 INFO  Anonymous Usage D

### try this query 
`MATCH 
  (v:SequenceVariant)-[]->(p:Patient),
  (v)-[]->(g:Gene),
  (c:CopyNumberAlteration)-[]->(p),
  (c)-[]->(g)
WHERE 
  v <> c
RETURN v, c, p, g;`

(Actually the WHERE condition isn't necessary)

#!/usr/bin/env python3
import argparse
import logging
import pandas as pd
import yaml

from biocypher import BioCypher
import decider_genetics.adapters.preprocessing_dataframes as preprocess_df
import decider_genetics.adapters.preprocessing_variants as preprocess_snv
import ontoweaver

if __name__ == "__main__":

    usage = f"Extract nodes and edges from CSV tables from synthetic clinical, copy number variations and single nucleotide variants and prepare a knowledge graph import script."
    parser = argparse.ArgumentParser(
        description=usage)

    parser.add_argument("-cli", "--synthetic_clinical", metavar="CSV", nargs="+",
                        help="Extract from the synthetic clinical CSV file.")

    parser.add_argument("-cna", "--synthetic_cna", metavar="CSV", nargs="+",
                        help="Extract from the synthetic copy number alterations CSV file.")

    parser.add_argument("-snv", "--synthetic_variants", metavar="CSV", nargs="+",
                        help="Extract from the synthetic single nucleotide variants CSV file.")
    
    parser.add_argument("-o", "--oncokb", metavar="CSV", nargs="+",
                        help="Extract from an OncoKB CSV file.")
    
    levels = {
        "DEBUG": logging.DEBUG,
        "INFO": logging.INFO,
        "WARNING": logging.WARNING,
        "ERROR": logging.ERROR,
        "CRITICAL": logging.CRITICAL
    }

    parser.add_argument("-v", "--verbose", choices = levels.keys(), default = "WARNING",
                        help="Set the verbose level (default: %(default)s).")
    
    asked = parser.parse_args()
    bc = BioCypher(
        biocypher_config_path="config/biocypher_config.yaml",
        schema_config_path="config/schema_config.yaml"
    )

    # Actually extract data.
    nodes = []
    edges = []

    data_mappings = {}

    # Extract from databases not requiring preprocessing.
    if asked.synthetic_clinical:
        logging.info(f"Weave synthetic clinical data...")
        
        clinical_df = pd.read_csv(asked.synthetic_clinical[0], sep=';')
        preprocessed_clinical_df = preprocess_df.preprocess_clinical(clinical_df)

        mapping_file = "./decider_genetics/adapters/clinical.yaml"
        with open(mapping_file) as fd:
            conf = yaml.full_load(fd)

        adapter = ontoweaver.tabular.extract_all(df=preprocessed_clinical_df, config=conf,separator = None, affix= "none")

        nodes += adapter.nodes
        edges += adapter.edges

        logging.info(f"Wove Clinical: {len(nodes)} nodes, {len(edges)} edges.")

    if asked.synthetic_cna:
        logging.info(f"Weave synthetic copy number alterations data")
        
        cna_df = pd.read_csv(asked.synthetic_cna[0], sep='\t')
        preprocessed_cna_df = preprocess_df.preprocess_cna(cna_df)

        mapping_file = "./decider_genetics/adapters/cna_genes.yaml"
        with open(mapping_file) as fd:
            conf = yaml.full_load(fd)

        adapter = ontoweaver.tabular.extract_all(df=preprocessed_cna_df, config=conf,separator = None, affix= "none")

        nodes += adapter.nodes
        edges += adapter.edges

        logging.info(f"Wove CNAs: {len(nodes)} nodes, {len(edges)} edges.")

    if asked.synthetic_variants:
        logging.info(f"Weave synthetic variants data")

        variants_df = pd.read_csv(asked.synthetic_variants[0], sep='\t')
        preprocessed_variants_df = preprocess_snv.preprocess_variants(variants_df)

        mapping_file = "./decider_genetics/adapters/variants.yaml"
        with open(mapping_file) as fd:
            conf = yaml.full_load(fd)

        adapter = ontoweaver.tabular.extract_all(df=preprocessed_variants_df, config=conf,separator = None, affix= "none")

        nodes += adapter.nodes
        edges += adapter.edges

        logging.info(f"Wove variants: {len(nodes)} nodes, {len(edges)} edges.")
    
    if asked.oncokb:
        logging.info(f"Weave OncoKB data...")
        oncokb_df = pd.read_csv(asked.oncokb[0], sep='\t')
        preprocessed_oncokb_df = preprocess_df.preprocess_oncokb(oncokb_df)

        mapping_file = "./decider_genetics/adapters/oncokb.yaml"
        with open(mapping_file) as fd:
            conf = yaml.full_load(fd)

        adapter = ontoweaver.tabular.extract_all(df=preprocessed_oncokb_df, config=conf,separator = None, affix= "none")

        nodes += adapter.nodes
        edges += adapter.edges

        logging.info(f"Wove OncoKB: {len(nodes)} nodes, {len(edges)} edges.")

    # # Write everything.
    # n, e = ontoweaver.extract(data_mappings)
    # nodes += n
    # edges += e

    import_file = ontoweaver.reconciliate_write(nodes, edges, "config/biocypher_config.yaml", "config/schema_config.yaml", separator=", ")
    # bc.write_schema_info(as_node=True)

    print(import_file)

In [ ]:
! pip install ipywidgets jupyterlab_widgets widgetsnbextension

## Broken attempt to set-up an interactive binary-choice tree that allows the user, by answreing questions about its datasets and goals in the skg construction, to be guided to an illustrational example 

In [ ]:
! pip install ipywidgets jupyterlab_widgets widgetsnbextension

In [ ]:
import ipywidgets as widgets
widgets.IntSlider()

In [ ]:
! jupyter lab build

In [ ]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=["Option 1", "Option 2"],
    value="Option 1",
    description="Choose:"
)

button = widgets.Button(description="Go")

def on_click(b):
    print(f"You chose: {dropdown.value}")

button.on_click(on_click)

display(dropdown, button)


In [ ]:
tree = {
    "Do you have structured data?": {
        "Yes": {
            "Is the schema stable?": {
                "Yes": "Example: SKG with fixed ontology and stable DB schema",
                "No": "Example: OntoWeaver mapping with dynamic schema detection"
            }
        },
        "No": {
            "Is the data semi-structured (e.g., XML, JSON)?": {
                "Yes": "Example: OntoWeaver on XML product feeds",
                "No": "Example: NLP-based SKG from unstructured text"
            }
        }
    }
}


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def traverse_tree(tree_node):
    clear_output()
    
    if isinstance(tree_node, str):
        print(tree_node)
        return

    question = list(tree_node.keys())[0]
    options = list(tree_node[question].keys())
    dropdown = widgets.Dropdown(options=options, description=question)
    button = widgets.Button(description="Next")

    def on_button_clicked(b):
        user_choice = dropdown.value
        next_node = tree_node[question][user_choice]
        traverse_tree(next_node)

    display(dropdown, button)
    button.on_click(on_button_clicked)

traverse_tree(tree)


In [ ]:
import sys
print(sys.executable)

In [ ]:
! which python

In [ ]:
! pyenv which python

In [ ]:
!{sys.executable} -m pip install ipywidgets